In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.stats import norm
from ploting import *
import seaborn as sns
%matplotlib inline

import torch
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
torch.manual_seed(123)
latexify(fig_width=10)

## Create dataset

In [2]:
def generate_data(samples=500000):
    data = []
    labels = []
    for i in range(samples):
        category = (np.random.choice([0, 1]), np.random.choice([0, 1]))
        if category == (1, 0):
            data.append([np.random.uniform(0.1, 1), 0])
            labels.append([1, 0, 1])
        if category == (0, 1):
            data.append([0, np.random.uniform(0.1, 1)])
            labels.append([0, 1, 0])
        if category == (0, 0):
            data.append([np.random.uniform(0.1, 1), np.random.uniform(0.1, 1)])
            labels.append([0, 0, 1])
    return train_test_split(np.vstack((data)), np.vstack((labels)), random_state=42)      

In [3]:
X_train, X_test, y_train, y_test = generate_data()

## Define model

In [4]:
class Classifier(torch.nn.Module):

    def __init__(self, nb_label):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Classifier, self).__init__()
        self.main = torch.nn.Sequential(torch.nn.Linear(2, 64),
                                  torch.nn.ReLU(),
                                  torch.nn.Linear(64, nlabel),)

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        
        return self.main(x)
        

In [5]:
def model_train(x_data, y_data, model, lrate):
    
    x_data = torch.from_numpy(x_data).float()
    y_data = torch.from_numpy(y_data).float()
    
    x_data = Variable(x_data)
    y_data = Variable(y_data)
    
    if torch.cuda.is_available():
        model = model.cuda()
        x_data = x_data.cuda()
        y_data = y_data.cuda()
        
    criterion = torch.nn.MultiLabelSoftMarginLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lrate)
    
    
    Epoch = []
    Loss  = []
 
    for epoch in range(50000):
        y_pred = model(x_data)

        loss = criterion(y_pred, y_data)
        Epoch.append(epoch)
        Loss.append(loss.data[0])
    
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    
    return Epoch, Loss 

In [6]:
nlabel = y_train.shape[1]
classifier = Classifier(nlabel)
print(classifier)

Classifier(
  (main): Sequential(
    (0): Linear(in_features=2, out_features=64)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=3)
  )
)


In [ ]:
epoch, loss = model_train(X_train, y_train, classifier, 0.01)

In [ ]:
plt.plot(epoch, loss, label="$lrate=0.1$")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss vs iterations")
plt.legend();